In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
n_words = 20000
word_dim = 300
n_filters = 64
n_prop = 651
data = './'

In [14]:
class CNN_Text(nn.Module):
    def __init__(self, input_dim, n_filters):
        super(CNN_Text, self).__init__()
        D = input_dim
        Ci = 1
        Co = n_filters
        Ks = [3, 4, 5]
        self.convs = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])
        self.fc = nn.Sequential(nn.Linear(n_filters * 3, 100), nn.Tanh())

    def forward(self, x):
        x = x.unsqueeze(1)  # (N, Ci, W, D)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs] # [(N, Co, W), ...]*len(Ks)
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x] # [(N, Co), ...]*len(Ks)
        x = torch.cat(x, 1)
        return self.fc(x)

In [15]:
def load_emb_matrix(vocab_size, emb_size, data):
    
    embedding_weights = {}
    f = open('../glove.42B.{}d.txt'.format(emb_size), 'r')
    
    loop = tqdm(f)
    loop.set_description('Load Glove')
    
    for line in loop:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        
    embedding_weights[word] = coefs
    
    f.close()
    
    print('Total {} word vectors in Glove.'.format(len(embedding_weights)))

    embedding_matrix = np.random.uniform(-1.0, 1.0, (vocab_size, emb_size))
    
    embedding_matrix[0, :] = np.zeros(emb_size)

    oov_count = 0
    
    vocab = load_vocabulary(os.path.join(data, 'word_vocab.pkl'))
    
    for word, i in vocab.items():
        embedding_vector = embedding_weights.get(word)
        
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        oov_count += 1
        
    print('Number of OOV words: %d' % oov_count)

    return embedding_matrix

In [16]:
    class NeuralNet(torch.nn.Module):

        def __init__(self):
            super(NeuralNet, self).__init__()

            #embedding layer
            self.word_embed = nn.Embedding(n_words, word_dim, max_norm=1, padding_idx=0)

            self.word_embed.weight = nn.Parameter(
              torch.from_numpy(load_emb_matrix(n_words, word_dim, data)).float()
            )

            self.CNN = CNN_Text(word_dim, n_filters)

            self.RNN = nn.GRU(input_size=word_dim, hidden_size=50, bidirectional=True, batch_first=True)

            self.info_proj = nn.Sequential(nn.Linear(n_prop, 100), nn.Tanh())

            self.projection = nn.Linear(300, 100)


        def forward(self, x):

            # x = [info, desc, short desc]
            info = x['info']
            info_feature = self.info_proj(info.float())

            desc = x['desc'][0]
            desc_feature = self.CNN(self.word_embed(desc))

            short_desc = x['short_desc'][0]
            out, hidden = self.RNN(self.word_embed(short_desc))
            short_desc_feature = torch.mean(out, dim=1)

            feature = torch.cat([info_feature, short_desc_feature, desc_feature], -1)
            return self.projection(feature)

In [17]:
net = NeuralNet()
net.cuda()

FileNotFoundError: [Errno 2] No such file or directory: '../glove.42B.300d.txt'